In [1]:
import os
import os.path
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as pyplot

In [2]:

crop_yield=pd.read_csv("data/BRC_TxF_yield_1970-2015.csv")

input_costs=pd.read_csv("data/TxF_Econ_model_output.csv")

corn_price=pd.read_csv("data/corn_price_USDANASS.csv")


In [3]:
crop_yield.head()

,Plot_Idnum,Old_plot_num,Year,Rep,Treatment,Treatment_st,Fert,Fert_st,Tillage,Tillage_st,...,Crop,Rotation,Height_in,Lodging_pct,Pop1_30DAP_no_ac,Pop2_EOS_no_ac,Moisture_pct,Ears_no_ac,Barren_pct,Sampled_ear_bean_weight_g
0,101,101_1,1970,1,1,CT_ctrl,1,Control,1,Conv,...,Corn,CC,96.0,6.5,NaN,27000.0,NaN,NaN,12.9,99.8
1,102,102_1,1970,1,2,CT_N,2,N_only,1,Conv,...,Corn,CC,109.0,9.3,NaN,32600.0,NaN,NaN,13.3,117.9
2,103,103_1,1970,1,3,CT_NwNPK,3,NplusNPKst,1,Conv,...,Corn,CC,108.0,7.6,NaN,28800.0,NaN,NaN,9.1,136.1
3,104,104_1,1970,1,4,CT_NPK,4,NPK,1,Conv,...,Corn,CC,115.0,13.6,NaN,25700.0,NaN,NaN,3.4,145.1
4,105,105_1,1970,1,5,CT_NPKwNPK,5,NPKplusNPKst,1,Conv,...,Corn,CC,118.0,9.7,NaN,31400.0,NaN,NaN,8.3,149.7


In [4]:
input_costs.head()

,Unnamed: 0,Year,rep,Treatment_num,fert,tillage,Treatment_st,Fert_st,Tillage_st,old_plot_num,...,P_cost,K_cost,Fert_cost,Herb_cost,Lime_cost,Total_overhead,Crop_rev,Profit,cum.profit.cont,cum.profit.rot
0,1,1970,1,1,1,1,CT_ctrl,Control,Conv,101_1,...,0.000000,0.000000,NaN,0.0,inf,NaN,NaN,NaN,0,0
1,2,1970,1,2,2,1,CT_N,N_only,Conv,102_1,...,0.000000,0.000000,NaN,0.0,inf,NaN,NaN,NaN,0,0
2,3,1970,1,3,3,1,CT_NwNPK,NplusNPKst,Conv,103_1,...,33.018713,8.419772,NaN,0.0,inf,NaN,NaN,NaN,0,0
3,4,1970,1,4,4,1,CT_NPK,NPK,Conv,104_1,...,33.018713,42.098860,NaN,0.0,inf,NaN,NaN,NaN,0,0
4,5,1970,1,5,5,1,CT_NPKwNPK,NPKplusNPKst,Conv,105_1,...,33.018713,42.098860,NaN,0.0,inf,NaN,NaN,NaN,0,0


In [5]:
corn_price.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2014,MARKETING YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",TOTAL,NOT SPECIFIED,3.70,NaN
1,SURVEY,2014,YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",TOTAL,NOT SPECIFIED,4.11,NaN
2,SURVEY,2013,MARKETING YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",TOTAL,NOT SPECIFIED,4.46,NaN
3,SURVEY,2013,YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",TOTAL,NOT SPECIFIED,6.15,NaN
4,SURVEY,2012,MARKETING YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,CORN,"CORN, GRAIN - PRICE RECEIVED, MEASURED IN $ / BU",TOTAL,NOT SPECIFIED,6.89,NaN


In [15]:
corn_price = corn_price.loc[(corn_price['Period'] == "MARKETING YEAR") & (corn_price['Commodity']=="CORN")].copy()
corn_price=corn_price[["Year", "Value"]]
corn_price.head()

,Year,Value
0,2014,3.70
2,2013,4.46
4,2012,6.89
6,2011,6.22
8,2010,5.18


In [13]:
total= crop_yield.merge(input_costs, on=["Plot_Idnum", "Year"], how="inner")
total=total[["Plot_Idnum", "Year", "Fert_st_x","Tillage_st_x", "Yield_bu_ac", "mach_cost", "diesel_cost", "labor_cost", "P_cost", "K_cost"]]
total["input_cost"]=total[["mach_cost", "diesel_cost", "labor_cost", "P_cost", "K_cost"]].sum(axis=1)
total = total[total['Fert_st_x'] == "Control"].copy()
total.head()

,Plot_Idnum,Year,Fert_st_x,Tillage_st_x,Yield_bu_ac,mach_cost,diesel_cost,labor_cost,P_cost,K_cost,input_cost
0,101,1970,Control,Conv,55.4,24.786175,1.907663,5.645687,0.0,0.0,32.339525
5,106,1970,Control,Alt,31.5,1.838698,0.233591,0.649808,0.0,0.0,2.722098
10,111,1970,Control,Chisel,55.9,24.377575,1.778573,4.896497,0.0,0.0,31.052645
15,116,1970,Control,No_till,35.4,1.838698,0.233591,0.649808,0.0,0.0,2.722098
20,201,1970,Control,Conv,90.8,24.786175,1.907663,5.645687,0.0,0.0,32.339525


In [16]:
total= total.merge(corn_price, on=["Year"], how="inner")
total["income_per_ac"]=total["Yield_bu_ac"]*total["Value"]
total["profit"]=total["income_per_ac"]-total["input_cost"]
total["roi"]=total["profit"]/total["input_cost"]*100
total.head()

,Plot_Idnum,Year,Fert_st_x,Tillage_st_x,Yield_bu_ac,mach_cost,diesel_cost,labor_cost,P_cost,K_cost,input_cost,Value,income_per_ac,profit,roi
0,101,1970,Control,Conv,55.4,24.786175,1.907663,5.645687,0.0,0.0,32.339525,1.33,73.682,41.342475,127.838844
1,106,1970,Control,Alt,31.5,1.838698,0.233591,0.649808,0.0,0.0,2.722098,1.33,41.895,39.172902,1439.070487
2,111,1970,Control,Chisel,55.9,24.377575,1.778573,4.896497,0.0,0.0,31.052645,1.33,74.347,43.294355,139.422440
3,116,1970,Control,No_till,35.4,1.838698,0.233591,0.649808,0.0,0.0,2.722098,1.33,47.082,44.359902,1629.622071
4,201,1970,Control,Conv,90.8,24.786175,1.907663,5.645687,0.0,0.0,32.339525,1.33,120.764,88.424475,273.425398


In [17]:
total.to_csv("data/total.csv")